In [4]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.graph_objects as go
import plotly.express as px
from sklearn.datasets import load_breast_cancer, load_diabetes, load_wine, load_iris
from sklearn.feature_selection import SelectKBest, f_classif, mutual_info_classif, RFE
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score, train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score
from sklearn.preprocessing import StandardScaler
import pandas as pd
from scipy.spatial.distance import pdist, squareform
import networkx as nx

class AdvancedSpringFeatureSelector:
    def __init__(self, 
                 performance_weight=0.3,
                 diversity_weight=0.2,
                 strong_threshold=0.6,
                 selection_ratio=0.4,
                 iterations=150):
        
        self.performance_weight = performance_weight
        self.diversity_weight = diversity_weight
        self.strong_threshold = strong_threshold
        self.selection_ratio = selection_ratio
        self.iterations = iterations
        
        self.positions = None
        self.springs = []
        self.correlations = None
        self.feature_scores = None
        self.selected_features = []
        self.performance_metrics = {}
        self.feature_explanations = {}
    
    def calculate_performance_scores(self, X, y, problem_type='classification'):
        """Calculate multiple performance metrics for each feature"""
        n_features = X.shape[1]
        performance_scores = np.zeros(n_features)
        
        for i in range(n_features):
            if np.std(X[:, i]) > 0:
                if problem_type == 'classification':
                    # Correlation with target
                    try:
                        corr = abs(np.corrcoef(X[:, i], y)[0, 1])
                    except:
                        corr = 0
                    # Mutual information
                    try:
                        mi = mutual_info_classif(X[:, i].reshape(-1, 1), y, random_state=42)[0]
                    except:
                        mi = 0
                    # Simple model performance
                    try:
                        model = LogisticRegression(random_state=42)
                        cv_score = cross_val_score(model, X[:, i].reshape(-1, 1), y, cv=3, scoring='accuracy').mean()
                    except:
                        cv_score = 0
                    
                    # Combined performance score
                    performance_scores[i] = 0.4 * corr + 0.4 * mi + 0.2 * cv_score
                else:
                    # For regression
                    try:
                        corr = abs(np.corrcoef(X[:, i], y)[0, 1])
                        performance_scores[i] = corr
                    except:
                        performance_scores[i] = 0
        
        # Normalize
        max_score = np.max(performance_scores)
        if max_score > 0:
            performance_scores = performance_scores / max_score
        
        return performance_scores
    
    def calculate_diversity_scores(self, correlations, selected_mask=None):
        """Calculate diversity scores to avoid redundant features"""
        n_features = correlations.shape[0]
        diversity_scores = np.ones(n_features)
        
        if selected_mask is None:
            selected_mask = np.zeros(n_features, dtype=bool)
        
        for i in range(n_features):
            if not selected_mask[i]:
                # Penalize features that are too similar to already selected features
                similarity_to_selected = 0
                if np.any(selected_mask):
                    similarity_to_selected = np.max(correlations[i, selected_mask])
                
                # Also consider overall redundancy
                high_correlations = np.sum(correlations[i] > 0.7) - 1  # -1 for self
                
                diversity_score = (1.0 - similarity_to_selected) * (1.0 / (1.0 + high_correlations * 0.1))
                diversity_scores[i] = diversity_score
        
        # Normalize
        max_score = np.max(diversity_scores)
        if max_score > 0:
            diversity_scores = diversity_scores / max_score
        
        return diversity_scores
    
    def run_spring_simulation(self, correlations):
        """Run the enhanced spring simulation"""
        n_features = correlations.shape[0]
        
        # Initialize springs and positions
        self.initialize_enhanced_springs(correlations)
        self.initialize_positions(n_features)
        
        # Run simulation
        for iteration in range(self.iterations):
            self.update_dynamics_enhanced()
            
        return self.compute_spring_scores()
    
    def initialize_enhanced_springs(self, correlations):
        """Initialize springs with enhanced physics"""
        n_features = correlations.shape[0]
        self.springs = []
        
        for i in range(n_features):
            for j in range(i + 1, n_features):
                corr = abs(correlations[i, j])
                
                if corr > self.strong_threshold:
                    # Strong correlation - compressed spring
                    rest_length = 0.3 + 0.4 * (1 - (corr - self.strong_threshold) / (1 - self.strong_threshold))
                    strength = 0.3 * corr
                    spring_type = "compressed"
                    color = 'red'
                    width = 3 + 2 * corr
                    
                elif corr > 0.3:
                    # Moderate correlation
                    rest_length = 1.2
                    strength = 0.1
                    spring_type = "neutral" 
                    color = 'green'
                    width = 1.5
                    
                else:
                    # Weak correlation
                    rest_length = 2.0
                    strength = 0.05
                    spring_type = "extended"
                    color = 'blue'
                    width = 1
                
                self.springs.append({
                    'from': i, 'to': j, 'rest_length': rest_length,
                    'strength': strength, 'type': spring_type,
                    'correlation': corr, 'color': color, 'width': width
                })
    
    def initialize_positions(self, n_features):
        """Initialize positions in 3D space"""
        # Start with features in a sphere
        indices = np.arange(n_features)
        phi = np.arccos(1 - 2 * indices / n_features)
        theta = np.pi * (1 + 5**0.5) * indices
        
        radius = 3.0
        x = radius * np.sin(phi) * np.cos(theta)
        y = radius * np.sin(phi) * np.sin(theta)
        z = radius * np.cos(phi)
        
        self.positions = np.column_stack([x, y, z])
        self.velocities = np.zeros_like(self.positions)
        self.initial_positions = self.positions.copy()
    
    def update_dynamics_enhanced(self, dt=0.1, damping=0.9):
        """Enhanced dynamics update"""
        forces = self.compute_forces_enhanced()
        self.velocities += forces * dt
        self.velocities *= damping
        self.positions += self.velocities * dt
    
    def compute_forces_enhanced(self):
        """Compute enhanced forces with cluster preservation"""
        n_features = len(self.positions)
        forces = np.zeros_like(self.positions)
        
        # Spring forces
        for spring in self.springs:
            i, j = spring['from'], spring['to']
            pos_i, pos_j = self.positions[i], self.positions[j]
            
            delta = pos_j - pos_i
            distance = np.linalg.norm(delta)
            
            if distance > 0:
                displacement = distance - spring['rest_length']
                force_magnitude = spring['strength'] * displacement
                
                # Non-linear force for better convergence
                if abs(displacement) > 1.0:
                    force_magnitude *= 1.5
                
                force_direction = delta / distance
                
                forces[i] += force_direction * force_magnitude
                forces[j] -= force_direction * force_magnitude
        
        # Smart repulsion
        for i in range(n_features):
            for j in range(i + 1, n_features):
                delta = self.positions[j] - self.positions[i]
                distance = np.linalg.norm(delta)
                
                if distance > 0:
                    # Adaptive repulsion based on correlation
                    corr = abs(self.correlations[i, j]) if self.correlations is not None else 0
                    repulsion_strength = 0.1 / (1.0 + corr * 5)  # Less repulsion for correlated features
                    repulsion_force = repulsion_strength / (distance ** 2 + 0.1)
                    
                    force_direction = delta / distance
                    forces[i] -= force_direction * repulsion_force
                    forces[j] += force_direction * repulsion_force
        
        return forces
    
    def compute_spring_scores(self):
        """Compute spring dynamics scores"""
        n_features = len(self.positions)
        
        # Centrality score
        distances = np.linalg.norm(self.positions, axis=1)
        centrality_scores = 1.0 / (1.0 + distances)
        
        # Compression and connectivity scores
        compression_scores = np.zeros(n_features)
        connectivity_scores = np.zeros(n_features)
        
        for spring in self.springs:
            if spring['type'] == "compressed":
                i, j = spring['from'], spring['to']
                actual_distance = np.linalg.norm(self.positions[i] - self.positions[j])
                if actual_distance < spring['rest_length'] * 1.2:  # Actually compressed
                    compression_scores[i] += spring['correlation']
                    compression_scores[j] += spring['correlation']
                    connectivity_scores[i] += 1
                    connectivity_scores[j] += 1
        
        # Stability score
        movement = np.linalg.norm(self.positions - self.initial_positions, axis=1)
        stability_scores = 1.0 / (1.0 + movement)
        
        # Normalize scores
        centrality_scores = centrality_scores / (np.max(centrality_scores) + 1e-8)
        compression_scores = compression_scores / (np.max(compression_scores) + 1e-8)
        connectivity_scores = connectivity_scores / (np.max(connectivity_scores) + 1e-8)
        stability_scores = stability_scores / (np.max(stability_scores) + 1e-8)
        
        # Combine spring scores
        spring_scores = (
            0.3 * centrality_scores +
            0.4 * compression_scores +
            0.2 * connectivity_scores +
            0.1 * stability_scores
        )
        
        return spring_scores
    
    def select_features_advanced(self, X, y, problem_type='classification'):
        """Advanced feature selection with performance and diversity"""
        self.correlations = self.calculate_correlations(X)
        n_features = X.shape[1]
        n_select = max(1, int(n_features * self.selection_ratio))
        
        print("🚀 ADVANCED SPRING DYNAMICS FEATURE SELECTION")
        print("=" * 60)
        
        # Step 1: Run spring simulation
        spring_scores = self.run_spring_simulation(self.correlations)
        
        # Step 2: Calculate performance scores
        performance_scores = self.calculate_performance_scores(X, y, problem_type)
        
        # Step 3: Iterative selection with diversity
        selected_features = []
        selected_mask = np.zeros(n_features, dtype=bool)
        
        for round_num in range(n_select):
            # Calculate diversity scores based on current selection
            diversity_scores = self.calculate_diversity_scores(self.correlations, selected_mask)
            
            # Combined scores for this round
            combined_scores = (
                (1 - self.performance_weight - self.diversity_weight) * spring_scores +
                self.performance_weight * performance_scores +
                self.diversity_weight * diversity_scores
            )
            
            # Select best feature not already chosen
            available_scores = combined_scores.copy()
            available_scores[selected_mask] = -np.inf  # Exclude already selected
            
            best_feature = np.argmax(available_scores)
            selected_features.append(best_feature)
            selected_mask[best_feature] = True
            
            if round_num < 5:  # Only print first 5 for brevity
                print(f"   Round {round_num + 1}: Selected Feature {best_feature} "
                      f"(Spring: {spring_scores[best_feature]:.3f}, "
                      f"Perf: {performance_scores[best_feature]:.3f}, "
                      f"Div: {diversity_scores[best_feature]:.3f})")
        
        self.selected_features = selected_features
        self.feature_scores = combined_scores
        
        # Store performance metrics
        self.performance_metrics = {
            'spring_scores': spring_scores,
            'performance_scores': performance_scores,
            'diversity_scores': diversity_scores,
            'final_scores': combined_scores
        }
        
        print(f"✅ Selected {len(selected_features)} features with advanced criteria")
        return selected_features
    
    def calculate_correlations(self, data):
        """Calculate correlation matrix"""
        try:
            correlations = np.corrcoef(data.T)
            np.fill_diagonal(correlations, 0)
            return correlations
        except:
            try:
                correlations = 1 - squareform(pdist(data.T, metric='correlation'))
                np.fill_diagonal(correlations, 0)
                return np.nan_to_num(correlations)
            except:
                # Fallback: simple correlation calculation
                n_features = data.shape[1]
                correlations = np.zeros((n_features, n_features))
                for i in range(n_features):
                    for j in range(n_features):
                        if i != j:
                            try:
                                corr = np.corrcoef(data[:, i], data[:, j])[0, 1]
                                correlations[i, j] = corr
                            except:
                                correlations[i, j] = 0
                return correlations

# Test the fixed implementation
print("🔧 TESTING FIXED ADVANCED SPRING DYNAMICS")
print("=" * 50)

# Load a simple dataset first
from sklearn.datasets import make_classification
X, y = make_classification(n_samples=100, n_features=20, n_informative=5, random_state=42)

# Test the advanced selector
advanced_selector = AdvancedSpringFeatureSelector(selection_ratio=0.3)
selected_features = advanced_selector.select_features_advanced(X, y)

print(f"\n✅ Successfully selected {len(selected_features)} features: {selected_features}")

🔧 TESTING FIXED ADVANCED SPRING DYNAMICS
🚀 ADVANCED SPRING DYNAMICS FEATURE SELECTION
   Round 1: Selected Feature 11 (Spring: 0.878, Perf: 0.689, Div: 0.900)
   Round 2: Selected Feature 8 (Spring: 0.926, Perf: 0.433, Div: 1.000)
   Round 3: Selected Feature 7 (Spring: 0.357, Perf: 1.000, Div: 0.901)
   Round 4: Selected Feature 4 (Spring: 0.262, Perf: 0.951, Div: 0.897)
   Round 5: Selected Feature 10 (Spring: 0.288, Perf: 0.759, Div: 1.000)
✅ Selected 6 features with advanced criteria

✅ Successfully selected 6 features: [np.int64(11), np.int64(8), np.int64(7), np.int64(4), np.int64(10), np.int64(6)]


In [5]:
class PublicationReadySpringSelector(AdvancedSpringFeatureSelector):
    """Publication-ready implementation with comprehensive evaluation"""
    
    def __init__(self, **kwargs):
        super().__init__(**kwargs)
        self.evaluation_metrics = {}
        self.comparison_results = {}
    
    def comprehensive_evaluation(self, X, y, feature_names, dataset_name, problem_type='classification'):
        """Run comprehensive evaluation for publication"""
        print(f"📊 COMPREHENSIVE EVALUATION: {dataset_name}")
        print("=" * 60)
        
        # 1. Run advanced spring selection
        spring_features = self.select_features_advanced(X, y, problem_type)
        
        # 2. Compare with traditional methods
        comparison = self.compare_with_traditional_methods(X, y, len(spring_features), problem_type)
        
        # 3. Robustness analysis
        robustness = self.robustness_analysis(X, y, spring_features, problem_type)
        
        # Store results
        self.evaluation_metrics[dataset_name] = {
            'spring_features': spring_features,
            'comparison': comparison,
            'robustness': robustness,
            'feature_names': feature_names
        }
        
        return self.evaluation_metrics[dataset_name]
    
    def compare_with_traditional_methods(self, X, y, n_features, problem_type):
        """Compare with traditional feature selection methods"""
        comparison_results = {}
        
        # ANOVA F-test
        try:
            anova = SelectKBest(f_classif, k=n_features)
            anova.fit(X, y)
            anova_features = np.argsort(anova.scores_)[-n_features:].tolist()
            anova_performance = self.evaluate_features(X, y, anova_features, problem_type)
            comparison_results['ANOVA'] = {
                'performance': anova_performance,
                'features': anova_features
            }
        except Exception as e:
            print(f"   ANOVA failed: {e}")
        
        # Mutual Information
        try:
            mi = SelectKBest(mutual_info_classif, k=n_features)
            mi.fit(X, y)
            mi_features = np.argsort(mi.scores_)[-n_features:].tolist()
            mi_performance = self.evaluate_features(X, y, mi_features, problem_type)
            comparison_results['Mutual_Info'] = {
                'performance': mi_performance,
                'features': mi_features
            }
        except Exception as e:
            print(f"   Mutual Info failed: {e}")
        
        # Random Forest
        try:
            rf = RandomForestClassifier(n_estimators=100, random_state=42)
            rf.fit(X, y)
            rf_features = np.argsort(rf.feature_importances_)[-n_features:].tolist()
            rf_performance = self.evaluate_features(X, y, rf_features, problem_type)
            comparison_results['Random_Forest'] = {
                'performance': rf_performance,
                'features': rf_features
            }
        except Exception as e:
            print(f"   Random Forest failed: {e}")
        
        # RFE (Recursive Feature Elimination)
        try:
            if problem_type == 'classification':
                estimator = LogisticRegression(random_state=42)
            else:
                from sklearn.linear_model import LinearRegression
                estimator = LinearRegression()
            
            rfe = RFE(estimator=estimator, n_features_to_select=n_features)
            rfe.fit(X, y)
            rfe_features = np.where(rfe.support_)[0].tolist()
            rfe_performance = self.evaluate_features(X, y, rfe_features, problem_type)
            comparison_results['RFE'] = {
                'performance': rfe_performance,
                'features': rfe_features
            }
        except Exception as e:
            print(f"   RFE failed: {e}")
        
        return comparison_results
    
    def evaluate_features(self, X, y, features, problem_type):
        """Evaluate feature set performance"""
        if len(features) == 0:
            return 0
        
        if problem_type == 'classification':
            model = LogisticRegression(random_state=42)
            scores = cross_val_score(model, X[:, features], y, cv=5, scoring='accuracy')
            return scores.mean()
        else:
            from sklearn.linear_model import LinearRegression
            model = LinearRegression()
            scores = cross_val_score(model, X[:, features], y, cv=5, scoring='r2')
            return scores.mean()
    
    def robustness_analysis(self, X, y, features, problem_type, n_splits=5):
        """Analyze robustness across different data splits"""
        from sklearn.model_selection import KFold
        
        performances = []
        
        kf = KFold(n_splits=n_splits, shuffle=True, random_state=42)
        
        for train_idx, test_idx in kf.split(X):
            X_train, X_test = X[train_idx], X[test_idx]
            y_train, y_test = y[train_idx], y[test_idx]
            
            # Train model on selected features
            performance = self.evaluate_features(X_train, y_train, features, problem_type)
            performances.append(performance)
        
        robustness_metrics = {
            'mean_performance': np.mean(performances),
            'std_performance': np.std(performances),
            'min_performance': np.min(performances),
            'max_performance': np.max(performances)
        }
        
        return robustness_metrics
    
    def generate_publication_report(self):
        """Generate comprehensive publication report"""
        print("\n" + "="*70)
        print("📑 PUBLICATION-READY SPRING DYNAMICS FEATURE SELECTION REPORT")
        print("="*70)
        
        for dataset_name, results in self.evaluation_metrics.items():
            print(f"\n📊 DATASET: {dataset_name.upper()}")
            print("-" * 50)
            
            # Get spring dynamics performance
            spring_perf = self.evaluate_features(
                self.datasets[dataset_name]['X'], 
                self.datasets[dataset_name]['y'],
                results['spring_features'],
                self.datasets[dataset_name]['problem_type']
            )
            
            print(f"🎯 Spring Dynamics Performance: {spring_perf:.4f}")
            print(f"🔧 Selected {len(results['spring_features'])} features")
            
            print(f"\n📈 COMPARISON WITH TRADITIONAL METHODS:")
            for method_name, method_results in results['comparison'].items():
                perf_diff = spring_perf - method_results['performance']
                comparison_symbol = "✅" if perf_diff > 0 else "➖"
                print(f"   {comparison_symbol} {method_name:<15}: {method_results['performance']:.4f} "
                      f"(Difference: {perf_diff:+.4f})")
            
            print(f"\n🛡️  ROBUSTNESS ANALYSIS:")
            robustness = results['robustness']
            print(f"   Mean Performance: {robustness['mean_performance']:.4f}")
            print(f"   Standard Deviation: {robustness['std_performance']:.4f}")
            print(f"   Range: {robustness['min_performance']:.4f} - {robustness['max_performance']:.4f}")

# Create dataset tester
class RealWorldDatasetTester:
    def __init__(self):
        self.datasets = {}
    
    def load_real_datasets(self):
        """Load multiple real-world datasets"""
        print("📊 LOADING REAL-WORLD DATASETS")
        print("=" * 50)
        
        # Classification datasets
        datasets_to_load = {
            'breast_cancer': load_breast_cancer(),
            'wine': load_wine(), 
            'iris': load_iris()
        }
        
        for name, dataset in datasets_to_load.items():
            X, y = dataset.data, dataset.target
            
            # Handle feature names
            if hasattr(dataset, 'feature_names'):
                feature_names = list(dataset.feature_names)
            else:
                feature_names = [f'feature_{i}' for i in range(X.shape[1])]
            
            # Standardize features
            scaler = StandardScaler()
            X_scaled = scaler.fit_transform(X)
            
            self.datasets[name] = {
                'X': X_scaled,
                'y': y,
                'feature_names': feature_names,
                'problem_type': 'classification'
            }
            
            print(f"   {name}: {X.shape[1]} features, {X.shape[0]} samples")
        
        return self.datasets

# Run the complete publication-ready evaluation
print("\n📝 RUNNING PUBLICATION-READY EVALUATION")
print("=" * 60)

# Load datasets
tester = RealWorldDatasetTester()
tester.load_real_datasets()

# Create publication selector
publication_selector = PublicationReadySpringSelector(
    performance_weight=0.3,
    diversity_weight=0.2,
    selection_ratio=0.4
)

# Add datasets to publication selector for reporting
publication_selector.datasets = tester.datasets

# Test on multiple datasets
for dataset_name, data in tester.datasets.items():
    publication_selector.comprehensive_evaluation(
        data['X'], data['y'], 
        data['feature_names'], dataset_name,
        data['problem_type']
    )

# Generate final report
publication_selector.generate_publication_report()




📝 RUNNING PUBLICATION-READY EVALUATION
📊 LOADING REAL-WORLD DATASETS
   breast_cancer: 30 features, 569 samples
   wine: 13 features, 178 samples
   iris: 4 features, 150 samples
📊 COMPREHENSIVE EVALUATION: breast_cancer
🚀 ADVANCED SPRING DYNAMICS FEATURE SELECTION
   Round 1: Selected Feature 20 (Spring: 0.847, Perf: 0.983, Div: 0.500)
   Round 2: Selected Feature 7 (Spring: 0.729, Perf: 0.975, Div: 0.062)
   Round 3: Selected Feature 0 (Spring: 0.710, Perf: 0.893, Div: 0.016)
   Round 4: Selected Feature 3 (Spring: 0.684, Perf: 0.881, Div: 0.006)
   Round 5: Selected Feature 27 (Spring: 0.595, Perf: 0.980, Div: 0.041)
✅ Selected 12 features with advanced criteria
📊 COMPREHENSIVE EVALUATION: wine
🚀 ADVANCED SPRING DYNAMICS FEATURE SELECTION
   Round 1: Selected Feature 5 (Spring: 0.915, Perf: 0.752, Div: 0.900)
   Round 2: Selected Feature 6 (Spring: 0.766, Perf: 1.000, Div: 0.076)
   Round 3: Selected Feature 12 (Spring: 0.488, Perf: 0.800, Div: 0.346)
   Round 4: Selected Feature 1